# Introduction

The notebook is intended to predict the chance failure of a specific product code given its attributes and measurements.

This is the [dataset used](https://www.kaggle.com/competitions/tabular-playground-series-aug-2022).

In [ ]:
# Import Standard Libraries
import pandas as pd
import numpy as np

import plotly.express as px
from plotly.io import read_json

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit, cross_val_score, HalvingGridSearchCV, RandomizedSearchCV
from sklearn.metrics import precision_score, accuracy_score, recall_score, f1_score, roc_auc_score
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV

# Read Data

In [ ]:
# Read train & test data
train_data = pd.read_csv('./../../data/2022_08/train.csv')
test_data = pd.read_csv('./../../data/2022_08/test.csv')

In [ ]:
train_data.info()

In [ ]:
train_data.head()

# Exploratory Data Analysis

## id

In [ ]:
# Check for duplicates
print('ID duplicates: {}'.format(len(train_data[train_data['id'].duplicated()])))

## Product Code

In [ ]:
# Check for NaN values
print('Product Code NaN values: {}'.format(len(train_data[train_data['product_code'].isna()])))

In [ ]:
# Plot the Histogram of 'product_code' Distribution
figure = px.histogram(train_data, 
                      x='product_code', 
                      title='Product Code Distribution', 
                      labels={'product_code':'Product Code',
                              'count': 'Share'},
                      color_discrete_sequence=['darkgreen'],
                      height=500,
                      histnorm='',
                      template='plotly_dark')

figure.update_layout(yaxis_title='Share', 
                     font=dict(family="PT Sans", 
                               size=14), 
                     title_font=dict(family="PT Sans",
                                     size=30), 
                     title_x=0.5)

# Plot
figure.show()

## loading

In [ ]:
# Check for NaN values
print('Loading NaN values: {}'.format(len(train_data[train_data['loading'].isna()])))

In [ ]:
# Plot boxplot of 'loading'
figure = px.box(train_data, 
                x='loading', 
                title='Loading Distribution',
                color_discrete_sequence=['darkgreen'],
                template='plotly_dark')

figure.update_layout(font=dict(family="PT Sans", 
                               size=14), 
                     title_font=dict(family="PT Sans",
                                     size=30), 
                     title_x=0.5)

# Plot
figure.show()

Fill NaN values and clean outliers.

In [ ]:
# Plot the 'loading' distribution with respect to the 'product_code'
figure = px.box(train_data, 
                x='product_code', 
                y='loading',
                points='all',
                color='product_code',
                title='Loading Distribution per Product Code',
                color_discrete_sequence=px.colors.qualitative.Set3,
                height=500,
                template='plotly_dark')

figure.update_layout(font=dict(family="PT Sans", 
                               size=14), 
                     title_font=dict(family="PT Sans",
                                     size=30), 
                     title_x=0.5)

# Plot
figure.show()

## Attributes

In [ ]:
# Check for Nan Values
for i in range(4):

    print('Attribute {} NaN value count: {}'.format(i, len(train_data[train_data['attribute_' + str(i)].isna()])))

## Failure

In [ ]:
# Check 'failure' NaN values
print('Failure NaN values: {}'.format(len(train_data[train_data['failure'].isna()])))

In [ ]:
# Plot the Histogram of 'failure' Distribution
figure = px.histogram(train_data, 
                      x='failure', 
                      color_discrete_sequence=['darkgreen'],
                      title='Failure Distribution', 
                      labels={'failure':'Failure'},
                      height=500,
                      histnorm='percent',
                      template='plotly_dark')

figure.update_layout(yaxis_title='Share', 
                     font=dict(family="PT Sans", 
                               size=14), 
                     title_font=dict(family="PT Sans",
                                     size=30), 
                     title_x=0.5)

figure.show()

Imbalanced classes, classic!

# Data Preparation

## Features and Labels Definition

In [ ]:
# Define features and labels
categorical_features = ['product_code', 
                        'attribute_0', 
                        'attribute_1']

numerical_integer_features = ['attribute_2', 
                              'attribute_3', 
                              'measurement_0', 
                              'measurement_1', 
                              'measurement_2']

numerical_float_featues = ['loading', 
                           'measurement_3', 
                           'measurement_4', 
                           'measurement_5', 
                           'measurement_6', 
                           'measurement_7', 
                           'measurement_8', 
                           'measurement_9', 
                           'measurement_10', 
                           'measurement_11', 
                           'measurement_12', 
                           'measurement_13', 
                           'measurement_14', 
                           'measurement_15',
                           'measurement_16', 
                           'measurement_17']

label = ['failure']

## Numerical Features

In [ ]:
# Numerical features pipeline
numerical_features_pipeline = Pipeline(steps=[
    ('numerical_imputer', SimpleImputer(strategy='constant')),
    ('numerical_scaler', StandardScaler())
])

## Categorical Features

In [ ]:
# Categorical features pipeline
categorical_features_pipeline = Pipeline(steps=[
    ('categorical_imputer', SimpleImputer(strategy='most_frequent')),
    ('categorical_one_hot_encoder', OneHotEncoder(handle_unknown='ignore'))
])

## Bundel Data Preprocessing Steps

In [ ]:
# Bunlde data preprocessing steps
data_preprocessor = ColumnTransformer(
    transformers=[
        ('numerical_preprocessing', numerical_features_pipeline, numerical_integer_features + numerical_float_featues),
        ('categorical_preprocessing', categorical_features_pipeline, categorical_features)
    ])

# Train & Test Split

In [ ]:
# Define X and y for the training set
X = train_data.drop(['failure'], axis=1)
y = train_data['failure']

In [ ]:
# Split training data into train and validation
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
# Define the a Stratified K-fold Shuffle Splitter
stratified_kfold = StratifiedShuffleSplit(n_splits=8,
                                          test_size=.3, 
                                          random_state=0)

# Model Definition

In [ ]:
# Define the used metrics
metrics = ['accuracy', 'precision', 'recall', 'f1', 'roc_auc', 'cv_score']

In [ ]:
# Initialize DataFrame of model performance
performance = pd.DataFrame(columns=metrics)

## Logistic Regression

In [ ]:
# Model's Hyperparameters
max_iter_lr = 1000
solver_lr = 'lbfgs'
c_lr = 10

In [ ]:
# Define the model
model_lr = LogisticRegression(C=c_lr,
                              max_iter=max_iter_lr, 
                              solver=solver_lr)

# Define the pipeline
pipe_lr = Pipeline([
    ('data_preprocessing', data_preprocessor),
    ('logistic_regression', model_lr)
])

# Train the pipeline
pipe_lr.fit(X_train, 
            y_train)

# Predictions
predictions_pipe_lr = pipe_lr.predict(X_test)

# Model evaluation
accuracy_pipe_lr = round(accuracy_score(y_test, predictions_pipe_lr) * 100, 2)
precision_pipe_lr = round(precision_score(y_test, predictions_pipe_lr) * 100, 2)
recall_pipe_lr = round(recall_score(y_test, predictions_pipe_lr) * 100, 2)
f1_pipe_lr = round(f1_score(y_test, predictions_pipe_lr) * 100, 2)
roc_auc_pipe_lr = round(roc_auc_score(y_test, predictions_pipe_lr) * 100, 2)
cv_score_pipe_lr = round(cross_val_score(pipe_lr, X, y, cv=stratified_kfold).mean() * 100, 2)

print('Model accuracy: {}%'.format(accuracy_pipe_lr))
print('Model precision: {}%'.format(precision_pipe_lr))
print('Model recall: {}%'.format(recall_pipe_lr))
print('Model f1 score: {}%'.format(f1_pipe_lr))
print('Model ROC AUC: {}%'.format(roc_auc_pipe_lr))
print('Model Cross Validation score: {}%'.format(cv_score_pipe_lr))

In [ ]:
# Update 'performance' DataFrame
performance.loc['logistic_regression'] = [accuracy_pipe_lr, 
                                          precision_pipe_lr, 
                                          recall_pipe_lr, 
                                          f1_pipe_lr, 
                                          roc_auc_pipe_lr, 
                                          cv_score_pipe_lr]

## Logistic Regression - Cross Validation

Using the Scikit-Library with built-in Cross Validation ([referece](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegressionCV.html#sklearn.linear_model.LogisticRegressionCV)).

In [ ]:
# Model's Hyperparameters
max_iter_lr_cv = 1000
solver_lr_cv = 'lbfgs'
scoring_lr_cv = 'roc_auc'
c_lr_cv = 1

In [ ]:
# Define the model
model_lr_cv = LogisticRegressionCV(Cs=c_lr_cv,
                                   max_iter=max_iter_lr_cv,
                                   cv=stratified_kfold,
                                   scoring=scoring_lr_cv,
                                   solver=solver_lr_cv)

# Define the pipeline
pipe_lr_cv = Pipeline([
    ('data_preprocessing', data_preprocessor),
    ('logistic_regression', model_lr_cv)
])

# Train the pipeline
pipe_lr_cv.fit(X_train, 
               y_train)

# Predictions
predictions_pipe_lr_cv = pipe_lr_cv.predict(X_test)

# Model evaluation
accuracy_pipe_lr_cv = round(accuracy_score(y_test, predictions_pipe_lr_cv) * 100, 2)
precision_pipe_lr_cv = round(precision_score(y_test, predictions_pipe_lr_cv) * 100, 2)
recall_pipe_lr_cv = round(recall_score(y_test, predictions_pipe_lr_cv) * 100, 2)
f1_pipe_lr_cv = round(f1_score(y_test, predictions_pipe_lr_cv) * 100, 2)
roc_auc_pipe_lr_cv = round(roc_auc_score(y_test, predictions_pipe_lr_cv) * 100, 2)
cv_score_pipe_lr_cv = round(cross_val_score(pipe_lr_cv, X, y, cv=stratified_kfold).mean() * 100, 2)

print('Model accuracy: {}%'.format(accuracy_pipe_lr_cv))
print('Model precision: {}%'.format(precision_pipe_lr_cv))
print('Model recall: {}%'.format(recall_pipe_lr_cv))
print('Model f1 score: {}%'.format(f1_pipe_lr_cv))
print('Model ROC AUC: {}%'.format(roc_auc_pipe_lr_cv))
print('Model Cross Validation score: {}%'.format(cv_score_pipe_lr_cv))

In [ ]:
# Update 'performance' DataFrame
performance.loc['logistic_regression_cv'] = [accuracy_pipe_lr_cv,
                                             precision_pipe_lr_cv,
                                             recall_pipe_lr_cv,
                                             f1_pipe_lr_cv, 
                                             roc_auc_pipe_lr_cv, 
                                             cv_score_pipe_lr_cv]

## Logistic Regression - Randomized Grid Search

[Reference Documentation](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html#sklearn.model_selection.RandomizedSearchCV)

In [ ]:
# Hyperparameters' space
hyperparameters_space_lr_cv_rgs = {
    'logistic_regression__Cs': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
    'logistic_regression__penalty': ['l2', 'l1'],
    'logistic_regression__solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
    'logistic_regression__max_iter': [1000, 3000, 5000],
    'logistic_regression__class_weight': [None, 'balanced']
}

# Model's Hyperparameters
scoring_lr_cv_rgs = 'roc_auc'

In [ ]:
# Define RandomizedSearchCV function
def randomized_grid_search(estimator,
                           hyperparameters_space,
                           cross_validation,
                           n_jobs=2):
    """
    Function to perform the Randomized Grid Search and return the best parameters computed
    :param estimator: sklearn.linear_model estimator to optimise
    :param hyperparameters_space: Dictionary of hyperparameters space to explore
    :param cross_validation: Integer or sklearn.model_selection cross-validation generator
    :return Dictionary of best hyperparameters
    """
    
    # Perform an Halving Grid Search for hyperparameters tuning
    randomized_grid_search = RandomizedSearchCV(estimator=estimator,
                                                param_distributions=hyperparameters_space,
                                                n_jobs=n_jobs,
                                                cv=cross_validation,
                                                verbose=2)
    
    # Perform the search
    randomized_grid_search.fit(X_train,
                               y_train)
    
    print(f"Randomized Grid Search best score: {randomized_grid_search.best_score_ * 100}")
    print(f"Randomized  Search best parameters: {randomized_grid_search.best_params_}")

    return randomized_grid_search.best_params_

In [ ]:
# Define the model
model_lr_cv_rgs = LogisticRegressionCV(cv=stratified_kfold,
                                       scoring=scoring_lr_cv_rgs)

# Define the pipeline
pipe_lr_cv_rgs = Pipeline([
    ('data_preprocessing', preprocessor),
    ('logistic_regression', model_lr_cv_rgs)
])

# Perform the Hyperparameters' Tuning
best_params_lr_cv_rgs = randomized_grid_search(pipe_lr_cv_rgs,
                                               hyperparameters_space_lr_cv_rgs,
                                               stratified_kfold)

## Logistic Regression - Halving Grid Search

[Reference Documentation](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.HalvingGridSearchCV.html)

In [ ]:
# Hyperparameters' space
hyperparameters_space_lr_cv_hgs = {
    'logistic_regression__Cs': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
    'logistic_regression__penalty': ['l2', 'l1'],
    'logistic_regression__solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
    'logistic_regression__max_iter': [3000, 4000, 5000],
    'logistic_regression__class_weight': [None, 'balanced']
}

# Model's Hyperparameters
scoring_lr_cv_hgs = 'roc_auc'

In [ ]:
# Define HalvingGridSearchCV function
def halving_grid_search(estimator, 
                        hyperparameters_space, 
                        cross_validation, 
                        n_jobs=2):
    """
    Function to perform the Halving Grid Search and return the best parameters computed
    :param estimator: sklearn.linear_model estimator to optimise
    :param hyperparameters_space: Dictionary of hyperparameters space to explore
    :param cross_validation: Integer or sklearn.model_selection cross-validation generator
    :return Dictionary of best hyperparameters
    """
    
    # Perform an Halving Grid Search for hyperparameters tuning
    halving_grid_search = HalvingGridSearchCV(estimator=estimator, 
                                              param_grid=hyperparameters_space, 
                                              n_jobs=n_jobs, 
                                              cv=cross_validation, 
                                              verbose=0)
    
    # Perform the search
    halving_grid_search.fit(X_train, 
                            y_train)
    
    print(f"Halving Grid Search best score: {halving_grid_search.best_score_ * 100}")
    print(f"Halving  Search best parameters: {halving_grid_search.best_params_}")

    return halving_grid_search.best_params_

In [ ]:
# Define the model
model_lr_cv_hgs = LogisticRegressionCV(cv=stratified_kfold,
                                       scoring=scoring_lr_cv_hgs)

# Define the pipeline
pipe_lr_cv_hgs = Pipeline([
    ('data_preprocessing', preprocessor),
    ('logistic_regression', model_lr_cv_hgs)
])

# Perform the Hyperparameters' Tuning
best_params_lr_cv_hgs = halving_grid_search(pipe_lr_cv_hgs, 
                                            hyperparameters_space_lr_cv_hgs, 
                                            stratified_kfold)

In [ ]:
# Train the pipeline
pipe_lr_cv_hgs.fit(X_train,
                   y_train)

# Predictions
predictions_pipe_lr_cv = pipe_lr_cv.predict(X_test)

# Model evaluation
accuracy_pipe_lr_cv = round(accuracy_score(y_test, predictions_pipe_lr_cv) * 100, 2)
precision_pipe_lr_cv = round(precision_score(y_test, predictions_pipe_lr_cv) * 100, 2)
recall_pipe_lr_cv = round(recall_score(y_test, predictions_pipe_lr_cv) * 100, 2)
f1_pipe_lr_cv = round(f1_score(y_test, predictions_pipe_lr_cv) * 100, 2)
roc_auc_pipe_lr_cv = round(roc_auc_score(y_test, predictions_pipe_lr_cv) * 100, 2)
cv_score_pipe_lr_cv = round(cross_val_score(pipe_lr_cv, X, y, cv=stratified_kfold).mean() * 100, 2)

print('Model accuracy: {}%'.format(accuracy_pipe_lr_cv))
print('Model precision: {}%'.format(precision_pipe_lr_cv))
print('Model recall: {}%'.format(recall_pipe_lr_cv))
print('Model f1 score: {}%'.format(f1_pipe_lr_cv))
print('Model ROC AUC: {}%'.format(roc_auc_pipe_lr_cv))
print('Model Cross Validation score: {}%'.format(cv_score_pipe_lr_cv))

# Model Comparison

In [ ]:
# Plot the models' performance
figure = px.bar(performance,
                x=performance.index,
                y=performance.columns.values,
                labels={'index': 'Model', 'value': 'Performance'},
                barmode='group',
                title='Models Comparison',
                color_discrete_sequence=px.colors.qualitative.Set3,
                height=500,
                template='plotly_dark')

figure.update_layout(font=dict(family="PT Sans", 
                               size=14), 
                     title_font=dict(family="PT Sans",
                                     size=30), 
                     title_x=0.5)

# Save figure
figure.write_json("./plots/loading_product_code_distribution.json")

# Read & plot figure
read_json('./plots/loading_product_code_distribution.json').show()